In [13]:
!apt install -y swig cmake
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit1/requirements-unit1.txt
!apt-get update
!apt-get install -y python3-opengl
!apt install -y ffmpeg
!apt install -y xvfb
!pip3 install pyvirtualdisplay ipywidgets

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.18.4-2+deb11u1).
swig is already the newest version (4.0.2-1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
Hit:1 http://deb.debian.org/debian bullseye InRelease
Hit:2 http://deb.debian.org/debian-security bullseye-security InRelease
Hit:3 http://deb.debian.org/debian bullseye-updates InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-opengl is already the newest version (3.1.5+dfsg-1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.3.6-0+deb11u1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state i

In [ ]:
import os
os.kill(os.getpid(), 9)

In [4]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [4]:
import gymnasium

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

## 环境与模型配置

- observation Space
    - Horizontal pad coordinate (x)
    - Vertical pad coordinate (y)
    - Horizontal speed (x)
    - Vertical speed (y)
    - Angle
    - Angular speed
    - If the left leg contact point has touched the land (boolean)
    - If the right leg contact point has touched the land (boolean)
- action Space
    - Do nothing
    - Fire left orientation engine
    - Fire main engine
    - Fire right orientation engine
- reward
    - 着陆器离着陆场越近/越远，则增加/减少。 
    - 着陆器移动得越慢/越快，则增加/减少。 
    - 着陆器倾斜得越多（角度不是水平的），该值就越小。 
    - 每一条与地面接触的腿增加 10 分。 
    - 侧引擎点火每帧减少 0.03 点。 
    - 主机每点火一帧就减少 0.3 点。
    - 因坠毁或安全着陆而分别获得 -100 或 +100 分的额外奖励。
    - 如果某一集得分至少为 200 分，则**被视为解决方案。**

In [10]:
import gymnasium as gym

# First, we create our environment called LunarLander-v2
env = gym.make("LunarLander-v2")

# Then we reset this environment
observation, info = env.reset()

for _ in range(20):
  # Take a random action
  action = env.action_space.sample()
  print("Action taken:", action)

  # Do this action in the environment and get
  # next_state, reward, terminated, truncated and info
  observation, reward, terminated, truncated, info = env.step(action)

  # If the game is terminated (in our case we land, crashed) or truncated (timeout)
  if terminated or truncated:
      # Reset the environment
      print("Environment is reset")
      observation, info = env.reset()

env.close()

Action taken: 2
Action taken: 1
Action taken: 2
Action taken: 0
Action taken: 1
Action taken: 2
Action taken: 1
Action taken: 3
Action taken: 2
Action taken: 2
Action taken: 0
Action taken: 2
Action taken: 1
Action taken: 3
Action taken: 1
Action taken: 2
Action taken: 2
Action taken: 0
Action taken: 1
Action taken: 2


In [9]:
# We create our environment with gym.make("<name_of_the_environment>")
env = gym.make("LunarLander-v2")
env.reset()
print("_____OBSERVATION SPACE_____ \n")
print("Observation Space Shape", env.observation_space.shape)
print("Sample observation", env.observation_space.sample()) # Get a random observation

print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample()) # Take a random action

_____OBSERVATION SPACE_____ 

Observation Space Shape (8,)
Sample observation [-2.9232323  63.493168   -2.9861605   2.567606    2.0409708   0.95576024
  0.81009424  0.6645145 ]

 _____ACTION SPACE_____ 

Action Space Shape 4
Action Space Sample 3


## 创建环境和模型并训练

- env
    - 我们创建了 16 个环境的矢量化环境（一种将多个独立环境堆叠到单个环境中的方法），这样，**我们将在训练过程中获得更多样化的体验。**
- model 
    - 定义 PPO MlpPolicy 架构
        - 我们使用 MultiLayerPerceptron (MLPPolicy) 因为输入是一个向量
        - 如果我们有帧作为输入，我们将使用 CnnPolicy
    

In [15]:
import stable_baselines3 as sb3
sb3.common.utils.get_device()

device(type='cpu')

In [17]:
import torch 

In [22]:
torch.backends.mps.is_available()

False

In [16]:
# env
env = make_vec_env('LunarLander-v2', n_envs=16)

# model 
model = PPO(
    policy = 'MlpPolicy',
    env = env,
    n_steps = 1024,
    batch_size = 64,
    n_epochs = 4,
    gamma = 0.999,
    gae_lambda = 0.98,
    ent_coef = 0.01,
    verbose=1,device='mps')

# Train the agent
model.learn(total_timesteps=int(2e5))

# save
model_name = "models/ppo-LunarLander-v2"
model.save(model_name)

Using mps device


RuntimeError: PyTorch is not linked with support for mps devices

## 评估

- 使用monitor包装环境


In [7]:
import gymnasium as gym

In [8]:
# model 
model_name = "models/ppo-LunarLander-v2"
model = PPO.load(model_name)

# Evaluate the agent
eval_env = Monitor(gym.make("LunarLander-v2"))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=-78.76 +/- 106.63993246180424


## publish

In [1]:
from huggingface_hub import login 
from src.pwd import hf_token
login(token=hf_token, add_to_git_credential=True)


Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from huggingface_sb3 import load_from_hub, package_to_hub
# To log to our Hugging Face account to be able to upload models to the Hub.

In [ ]:
import gymnasium as gym

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env

from huggingface_sb3 import package_to_hub

# PLACE the variables you've just defined two cells above
# Define the name of the environment
env_id = "LunarLander-v2"

# Define the model architecture we used
model_architecture = "PPO"

## Define a repo_id
## repo_id is the id of the model repository from the Hugging Face Hub (repo_id = {organization}/{repo_name} for instance ThomasSimonini/ppo-LunarLander-v2
## CHANGE WITH YOUR REPO ID
repo_id = "AdamZh0u/ppo-LunarLander-v2" 

## Define the commit message
commit_message = "Upload PPO LunarLander-v2 trained agent"

# Create the evaluation env and set the render_mode="rgb_array"
eval_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])

# PLACE the package_to_hub function you've just filled here
package_to_hub(model=model, # Our trained model
               model_name=model_name, # The name of our trained model
               model_architecture=model_architecture, # The model architecture we used: in our case PPO
               env_id=env_id, # Name of the environment
               eval_env=eval_env, # Evaluation Environment
               repo_id=repo_id, # id of the model repository from the Hugging Face Hub (repo_id = {organization}/{repo_name} for instance ThomasSimonini/ppo-LunarLander-v2
               commit_message=commit_message)

## Load from hub

In [ ]:
from huggingface_sb3 import load_from_hub
repo_id = "Classroom-workshop/assignment2-omar" # The repo_id
filename = "ppo-LunarLander-v2.zip" # The model filename.zip

# When the model was trained on Python 3.8 the pickle protocol is 5
# But Python 3.6, 3.7 use protocol 4
# In order to get compatibility we need to:
# 1. Install pickle5 (we done it at the beginning of the colab)
# 2. Create a custom empty object we pass as parameter to PPO.load()
custom_objects = {
            "learning_rate": 0.0,
            "lr_schedule": lambda _: 0.0,
            "clip_range": lambda _: 0.0,
}

checkpoint = load_from_hub(repo_id, filename)
model = PPO.load(checkpoint, custom_objects=custom_objects, print_system_info=True)